In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy
%matplotlib inline

In [241]:
original_dataset = pd.read_csv('data/cancer_tpm_multiindexed.tsv', sep='\t', index_col=[0,1], header=[0,1])

In [242]:
# use only ENSG column IDs
original_dataset.columns = list(original_dataset.columns.get_level_values(0))

In [243]:
# Set output column as recurrence
output_col = original_dataset.index.get_level_values(1)

In [244]:
# Set index just to sample ID
original_dataset.index = list(original_dataset.index.get_level_values(0))

In [245]:
# Normalize the data????
norm = (original_dataset.loc['C1'] - original_dataset.loc['C1'].mean()).divide(original_dataset.loc['C1'].std())

In [246]:
# Add the output label
original_dataset['output'] = output_col

In [247]:
len(original_dataset[original_dataset.output == 1])

28

In [248]:
# Final dataset to do build the neural net.
original_dataset.head()

,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000283101,ENSG00000283103,ENSG00000283108,ENSG00000283110,ENSG00000283117,ENSG00000283118,ENSG00000283122,ENSG00000283123,ENSG00000283125,output
C1,7.071605,13.279391,0.000000,6.212355,34.038592,9.231355,32.811263,19.136966,12.631348,11.220057,...,0.0,4.430132,0.000000,0.0,6.856902,16.599238,22.357146,0.0,19.542796,1
C2,60.610797,47.424080,0.000000,2.218590,35.828348,4.395669,22.498053,0.000000,5.413170,48.083612,...,0.0,0.000000,48.996429,0.0,9.795095,0.000000,4.562460,0.0,0.000000,1
C3,58.255903,60.455497,23.040206,12.120963,55.926653,8.005046,13.657227,14.935300,16.430065,14.594347,...,0.0,0.000000,14.871406,0.0,13.378542,0.000000,12.463189,0.0,0.000000,1
C4,29.917356,6.482332,8.646690,6.065120,10.494273,0.000000,23.064209,18.683413,9.865584,8.215603,...,0.0,0.000000,33.486271,0.0,13.388781,0.000000,6.236364,0.0,0.000000,1
C5,24.500322,53.675826,20.456404,16.142519,14.482668,0.000000,59.112604,8.840272,55.432674,0.000000,...,0.0,0.000000,26.407358,0.0,19.797049,57.509813,3.688509,0.0,45.138720,1


In [500]:
highest_750_expressed_genes = list(original_dataset.mean().sort_values(ascending=False).head(750).index)

# Model construction based heavily on: https://github.com/philipobrien/colab-notebooks/blob/master/Non_Linear_Data_Classification.ipynb

# Split into train and test datasets

In [505]:
preselected_ensgs = pd.read_csv('data/preselectedList', names=['ENSG'])
ensgs_subset = list(preselected_ensgs['ENSG'])

In [685]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
sns.set()

# This never changes...
MANUAL_SEED = 2
torch.manual_seed(MANUAL_SEED)

# Model methods definition 
class Model(nn.Module):
    def __init__(self, input_size, H1, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, H1)
        self.linear2 = nn.Linear(H1, output_size)
    
    def forward(self, x):
        x = torch.sigmoid(self.linear(x))
        x = torch.sigmoid(self.linear2(x))
        return x
    
    def predict(self, x):
        pred = self.forward(x)
        return pred

# Split the datase for cross-validation purposes, 2/3 training, 1/3 testing
def split_dataset(original_dataset, fraction_test, random_seed, num_features=1000):
    train, test, train_labels, test_labels = train_test_split(original_dataset, 
                                                              output_col, 
                                                              stratify = output_col,
                                                              test_size=fraction_test,
                                                              random_state = SPLIT_SEED)
    
    #print('columns used: {}'.format(train.columns))
    
    print('Using only the first {} genes...'.format(num_features))
    X = np.array(train[train.columns[0:num_features]])
    y = np.array(train.output)

    # Tensorify the input and output arrays
    x_data = torch.FloatTensor(X)
    y_data = torch.FloatTensor(y.reshape(len(y), 1))
    return x_data, y_data, train_labels, test, test_labels

# Loss function information: 
# https://medium.com/udacity-pytorch-challengers/a-brief-overview-of-loss-functions-in-pytorch-c0ddb78068f7
def run_model(x_data, y_data, test, num_features, hidden_layer_size=4, learning_rate=0.0001, epochs=5000):
    model = Model(num_features, hidden_layer_size, 1)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    losses = []
    for i in range(epochs):
        # Make predictions based on current weights in model
        y_pred = model.forward(x_data)

        # Calculate new loss
        loss = criterion(y_pred, y_data)
        losses.append(loss.item())

        if i%10000==0:
            print(f"epoch: {i}, loss: {loss.item()}")

        # Calculate gradient and back-propagate to calculate the new weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Find probabilities on the test set
    X_test = np.array(test[test.columns[0:num_features]])
    test_tensor = torch.FloatTensor(X_test)

    probs = []
    for t in test_tensor:
        probs.append(model.predict(t))
    probs = list([float(t[0]) for t in probs])
    
    return losses, probs



In [694]:
from collections import defaultdict

def plot_average_loss_and_auc_per_layer_size(losses_list, probs_list, test_labels_list, plot_title_prefix):
    # Figure out average loss and average AUC for each layer size
    i = 0

    layer_to_losses = {}
    layer_to_average_losses = {}
    layer_to_indiv_probs = defaultdict(lambda:[])
    layer_to_probs = {}
    layer_to_average_probs = {}
    layer_to_test_labels = defaultdict(lambda:[])

    for split_seed in split_seeds:
        for hidden_layer_size in hidden_layer_sizes:
            next_loss = losses_list[i]
            next_probs = probs_list[i]
            next_test_labels = test_labels_list[i]

            # thin out the losses by only keeping every 1000th entry
            thin_loss = [next_loss[j] for j,l in enumerate(next_loss) if j % 1000 == 0]

            # Keep running sum of the losses and probabilities at each epoch 
            if hidden_layer_size in layer_to_losses:
                addition = [thin_loss[j] + v for j,v in enumerate(layer_to_losses[hidden_layer_size])]
                layer_to_losses[hidden_layer_size] = addition
            else:
                layer_to_losses[hidden_layer_size] = thin_loss

            if hidden_layer_size in layer_to_probs:
                addition = [next_probs[j] + v for j,v in enumerate(layer_to_probs[hidden_layer_size])]
                layer_to_probs[hidden_layer_size] = addition
            else:
                layer_to_probs[hidden_layer_size] = next_probs

            layer_to_indiv_probs[hidden_layer_size].append(next_probs)
            layer_to_test_labels[hidden_layer_size].append(next_test_labels)

            i = i + 1

    # Now divide by the number of epochs to get the average of the loss/probs at each epoch for each hidden layer size
    for hidden_layer_size, losses_per_size in layer_to_losses.items():
        layer_to_average_losses[hidden_layer_size] = [l/len(split_seeds) for l in losses_per_size]

    for hidden_layer_size, probs_per_size in layer_to_probs.items():
        layer_to_average_probs[hidden_layer_size] = [l/len(split_seeds) for l in probs_per_size]

    # Plot average losses with various hidden layer sizes
    plt.figure(figsize=(14,8))
    for k,v in layer_to_average_losses.items():
        plt.plot(v)
    plt.legend(['{} nodes'.format(k) for k in layer_to_average_losses.keys()])
    plt.title('{}: Average loss with various hidden layer sizes'.format(plot_title_prefix))
    plt.xlabel('Epochs (thousands)')
    plt.ylabel('Cross-entropy loss')

    layer_to_roc_aucs = defaultdict(lambda:[])
    for layer_size,v in layer_to_indiv_probs.items():
        test_labels = layer_to_test_labels.get(layer_size)

        for j, probability in enumerate(v):
            auc = roc_auc_score(test_labels[j], probability)
            layer_to_roc_aucs[layer_size].append(auc)


    import seaborn as sns, operator as op

    plt.figure(figsize=(14,8))
    # sort keys and values together
    sorted_keys, sorted_vals = list(layer_to_roc_aucs.keys()), list(layer_to_roc_aucs.values())

    # almost verbatim from question
    sns.set(context='notebook', style='whitegrid')
    sns.boxplot(data=sorted_vals, width=.18)
    sns.swarmplot(data=sorted_vals, size=6, edgecolor="black", linewidth=.9)

    # category labels
    plt.xticks(plt.xticks()[0], sorted_keys)
    plt.title('{}: ROC AUC at replicates of various hidden layer sizes'.format(plot_title_prefix))
    plt.ylabel('ROC AUC')
    plt.xlabel('Number of nodes in hidden layer')
    plt.show()
    

# Run the model using just the first 750 genes as a baseline to compare the preselected 750 breast cancer-associated genes against

In [733]:
# Variables used in all the different runs...
hidden_layer_sizes = [2, 4, 8, 16]
learning_rate = 0.0001
epochs = 100000
fraction_test = 0.33
split_seeds = range(1,15)

In [730]:
x_data, y_data, train_labels, test, test_labels = split_dataset(original_dataset, fraction_test, split_seed, 750)
len(x_data), len(y_data), len(train_labels), len(test), len(test_labels)

Using only the first 750 genes...


(64, 64, 64, 32, 32)

In [ ]:
from random import sample 

# Variables
num_genes = 750

test = None
all_losses = []
all_probs = []
all_test_labels = []

for split_seed in split_seeds:
    for hidden_layer_size in hidden_layer_sizes:
        print('Next run: split seed: {}, hidden layer size: {}...'.format(split_seed, hidden_layer_size))
        num_features = num_genes
        x_data, y_data, train_labels, test, test_labels = split_dataset(original_dataset, fraction_test, split_seed, num_features)
        losses, probs = run_model(x_data, y_data, test, num_features, hidden_layer_size, learning_rate, epochs)
        all_losses.append(losses)
        all_probs.append(probs)
        print('test labels length', len(test_labels))
        all_test_labels.append(test_labels)
        

In [ ]:
plot_average_loss_and_auc_per_layer_size(all_losses,
                                         all_probs, 
                                         all_test_labels, 
                                         'Random subset of 750 genes (first 750)')

# Now do it with just the 750 pre-selected genes

In [734]:
print(len(ensgs_subset))
preselected_dataset = original_dataset[ensgs_subset+['output']]
len(preselected_dataset.columns)

x_data, y_data, train_labels, test, test_labels = split_dataset(preselected_dataset, fraction_test, split_seed, 750)
len(x_data), len(y_data), len(train_labels), len(test), len(test_labels)

750
Using only the first 750 genes...


(64, 64, 64, 32, 32)

In [ ]:
# Variables
num_genes = len(ensgs_subset)


all_preselected_losses = []
all_preselected_probs = []
all_preselected_test_labels = []


for split_seed in split_seeds:
    for hidden_layer_size in hidden_layer_sizes:
        print('Next run: split seed: {}, hidden layer size: {}...'.format(split_seed, hidden_layer_size))
        num_features = num_genes
        x_data, y_data, train_labels, test, test_labels = split_dataset(original_dataset, fraction_test, split_seed, num_features)
        losses, probs = run_model(x_data, y_data, test, num_features, hidden_layer_size, learning_rate, epochs)
        all_preselected_losses.append(losses)
        all_preselected_probs.append(probs)
        print('test labels length', len(test_labels))
        all_preselected_test_labels.append(test_labels)
        


Next run: split seed: 1, hidden layer size: 2...
Using only the first 750 genes...
epoch: 0, loss: 0.6907444596290588
epoch: 10000, loss: 0.22866187989711761
epoch: 20000, loss: 0.13029073178768158
epoch: 30000, loss: 0.08345133811235428
epoch: 40000, loss: 0.05694384500384331
epoch: 50000, loss: 0.040585171431303024
epoch: 60000, loss: 0.029980510473251343
epoch: 70000, loss: 0.022349335253238678
epoch: 80000, loss: 0.01657821238040924
epoch: 90000, loss: 0.012230243533849716
test labels length 32
Next run: split seed: 1, hidden layer size: 4...
Using only the first 750 genes...
epoch: 0, loss: 0.9729025959968567
epoch: 10000, loss: 0.3856033682823181
epoch: 20000, loss: 0.222087562084198
epoch: 30000, loss: 0.09735608845949173
epoch: 40000, loss: 0.0829474925994873
epoch: 50000, loss: 0.07746990770101547
epoch: 60000, loss: 0.07593949139118195
epoch: 70000, loss: 0.07533413916826248
epoch: 80000, loss: 0.07509239763021469
epoch: 90000, loss: 0.07499512284994125
test labels length 32


In [ ]:
"""
pd.DataFrame([all_losses,
              all_probs,
              all_preselected_losses, 
              all_preselected_probs], index=['losses', 'probs', 'preselected_losses', 'preselected_probs'])\
.to_csv('losses_and_probs_first_two_steps.tsv', sep='\t', header=True)
"""

In [ ]:
plot_average_loss_and_auc_per_layer_size(all_preselected_losses,
                                         all_preselected_probs, 
                                         all_preselected_test_labels, 
                                         'Preselected 750 breast cancer-associated genes')

In [ ]:
"""
pd.DataFrame([all_losses,
              all_probs,
              all_preselected_losses, 
              all_preselected_probs
             ], index=['losses', 'probs', 'preselected_losses', 'preselected_probs', 'high_expression_losses', 'high_expression_probs'])\
.to_csv('losses_and_probs_first_three_steps.tsv', sep='\t', header=True)
"""

# Gene sets

In [ ]:
GSEA_KRAS_BREAST_UP_V1_UP = list(pd.read_csv('data/GSEA_KRAS_BREAST_UP_V1_UP', names=['ENSG']).ENSG)
GSEA_VANTVEER_BREAST_CANCER_POOR_PROGNOSIS = list(pd.read_csv('data/GSEA_VANTVEER_BREAST_CANCER_POOR_PROGNOSIS', names=['ENSG']).ENSG)
Cancer_census_genes = list(pd.read_csv('data/Cancer_census_genes', names=['ENSG']).ENSG)


# KRAS BREAST UP Gene set

In [ ]:
len(GSEA_KRAS_BREAST_UP_V1_UP)

In [ ]:
kras_dataset = original_dataset[[c for c in GSEA_KRAS_BREAST_UP_V1_UP if c in original_dataset.columns] + ['output']]
len(kras_dataset.columns)

In [ ]:
kras_dataset.head()

In [ ]:
# Variables
num_genes = len(kras_dataset.columns) - 1


all_kras_losses = []
all_kras_probs = []
all_kras_test_labels = []

for split_seed in split_seeds:
    for hidden_layer_size in hidden_layer_sizes:
        print('Next run: split seed: {}, hidden layer size: {}...'.format(split_seed, hidden_layer_size))
        num_features = num_genes
        x_data, y_data, train_labels, test, test_labels = split_dataset(kras_dataset, fraction_test, split_seed, num_features)
        losses, probs = run_model(x_data, y_data, test, num_features, hidden_layer_size, learning_rate, epochs)
        all_kras_losses.append(losses)
        all_kras_probs.append(probs)
        print('test labels length', len(test_labels))
        all_kras_test_labels.append(test_labels)
        
        
        


In [ ]:
plot_average_loss_and_auc_per_layer_size(all_kras_losses,
                                         all_kras_probs, 
                                         all_kras_test_labels, 
                                         '139 genes from GSEA_KRAS_BREAST_UP_V1_UP gene set')


# Vantveer Gene set

In [ ]:
print(len(GSEA_VANTVEER_BREAST_CANCER_POOR_PROGNOSIS))
vantveer_dataset = original_dataset[[c for c in GSEA_VANTVEER_BREAST_CANCER_POOR_PROGNOSIS if c in original_dataset.columns] + ['output']]
len(vantveer_dataset.columns)

In [ ]:
# Variables
num_genes = len(vantveer_dataset.columns) - 1


all_vantveer_losses = []
all_vantveer_probs = []
all_vantveer_test_labels = []

for split_seed in split_seeds:
    for hidden_layer_size in hidden_layer_sizes:
        print('Next run: split seed: {}, hidden layer size: {}...'.format(split_seed, hidden_layer_size))
        num_features = num_genes
        x_data, y_data, train_labels, test, test_labels = split_dataset(vantveer_dataset, fraction_test, split_seed, num_features)
        losses, probs = run_model(x_data, y_data, test, num_features, hidden_layer_size, learning_rate, epochs)
        all_vantveer_losses.append(losses)
        all_vantveer_probs.append(probs)
        print('test labels length', len(test_labels))
        all_vantveer_test_labels.append(test_labels)
        
        
        


In [ ]:
plot_average_loss_and_auc_per_layer_size(all_vantveer_losses,
                                         all_vantveer_probs, 
                                         all_vantveer_test_labels, 
                                         '60 genes from GSEA_VANTVEER_BREAST_CANCER_POOR_PROGNOSIS gene set')


# Cancer gene census gene set

In [ ]:
print(len(Cancer_census_genes))
cgc_dataset = original_dataset[[c for c in Cancer_census_genes if c in original_dataset.columns] + ['output']]
len(cgc_dataset.columns)



In [ ]:
# Variables
num_genes = len(cgc_dataset.columns) - 1

all_cgc_losses = []
all_cgc_probs = []
all_cgc_test_labels = []

for split_seed in split_seeds:
    for hidden_layer_size in hidden_layer_sizes:
        print('Next run: split seed: {}, hidden layer size: {}...'.format(split_seed, hidden_layer_size))
        num_features = num_genes
        x_data, y_data, train_labels, test, test_labels = split_dataset(cgc_dataset, fraction_test, split_seed, num_features)
        losses, probs = run_model(x_data, y_data, test, num_features, hidden_layer_size, learning_rate, epochs)
        all_cgc_losses.append(losses)
        all_cgc_probs.append(probs)
        print('test labels length', len(test_labels))
        all_cgc_test_labels.append(test_labels)
        
        
        


In [ ]:
plot_average_loss_and_auc_per_layer_size(all_cgc_losses,
                                         all_cgc_probs, 
                                         all_cgc_test_labels, 
                                         '710 genes from Cancer Gene Census gene set')


In [ ]:
"""
pd.DataFrame([all_losses,
              all_probs,
              all_preselected_losses, 
              all_preselected_probs,
              all_kras_losses,
              all_kras_probs
             ], index=['losses', 'probs', 
                                             'preselected_losses', 'preselected_probs',
                                             'kras_losses', 'kras_probs'
                                            ])\
.to_csv('losses_and_probs_saved.tsv', sep='\t', header=True)
"""